In [66]:
from transformers import AutoTokenizer
import pandas as pd
import csv
import numpy as np
from glob import glob
from transformers import AutoModelForTokenClassification, DistilBertForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
import torch
from source.eval import evaluate_indices
from sklearn.metrics import classification_report

In [44]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-cased")

In [45]:
sent = "This is, a real; sentence. Not really Legal...\nBut it is\nok."
tok_sent = tokenizer(sent)
print(tok_sent)
tokens = tokenizer.convert_ids_to_tokens(tok_sent["input_ids"])
print(tokens)

{'input_ids': [101, 1188, 1110, 117, 170, 1842, 132, 5650, 119, 1753, 1541, 10800, 119, 119, 119, 1252, 1122, 1110, 21534, 119, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['[CLS]', 'This', 'is', ',', 'a', 'real', ';', 'sentence', '.', 'Not', 'really', 'Legal', '.', '.', '.', 'But', 'it', 'is', 'ok', '.', '[SEP]']


In [46]:
files = glob("./documents/lines/train/*.txt")
dataset_train = []
for file in files: 
    text = open(file, "r")
    original = text.readlines()
    for i in range(len(original)):
        original[i]= original[i].strip('\n')
        dataset_train += [original[i]]

In [47]:
files = glob("./documents/lines/dev/*.txt")
dataset_dev = []
for file in files: 
    text = open(file, "r")
    original = text.readlines()
    for i in range(len(original)):
        original[i]= original[i].strip('\n')
        dataset_dev += [original[i]]

In [48]:
files = glob("./documents/lines/test/*.txt")
dataset_test = []
for file in files: 
    text = open(file, "r")
    original = text.readlines()
    for i in range(len(original)):
        original[i]= original[i].strip('\n')
        dataset_test += [original[i]]

In [49]:
df = pd.read_csv("./dataset_v20230110.tsv", delimiter="	", quoting=csv.QUOTE_NONE, encoding='utf-8')
df.head()

,document,content,tag
0,ROSENBLATT v. BAER_MCL,MR. JUSTICE BRENNAN delivered the opinion of t...,Announcing function
1,ROSENBLATT v. BAER_MCL,A jury in New Hampshire Superior Court awarded...,Describing the adjudicated facts
2,ROSENBLATT v. BAER_MCL,Respondent alleged that the column contained d...,Describing procedural events
3,ROSENBLATT v. BAER_MCL,In the interval between the trial and the deci...,Describing the adjudicated facts
4,ROSENBLATT v. BAER_MCL,We there held that consistent with the First a...,Recalling a SCOTUS decision


In [61]:
def make_dataset(dataset, return_tensors=None):
    sentences = []
    ner_tags = []
    fin_tags = []
    tokenizeds = []

    for i in range(len(dataset)):
        tok_sent = tokenizer(dataset[i])
        #print(type(tok_sent))
        tokens = tokenizer.convert_ids_to_tokens(tok_sent["input_ids"])
        sentences += tokens[1:-1]
        n_tag = np.zeros(len(tokens), dtype=int)
        n_tag[-1] = 1
        ner_tags += n_tag.tolist()
        tok_sent["labels"] = n_tag
        #tokenizeds_dev+= [tok_sent]

    for i in range(0, len(sentences), (507)):
        if i + 507 > len(sentences):
            tokens = ['CLS'] + sentences[i:len(sentences)] + ['SEP']
            fin_tags = [-100] + ner_tags[i:len(sentences)] + [-100]
        else : 
            tokens = ['CLS'] + sentences[i:i + 507] + ['SEP']
            fin_tags = [-100] + ner_tags[i:i + 507] + [-100]
        #print(tokenizer.convert_tokens_to_string(tokens))
        sent = tokenizer.decode(tokenizer.convert_tokens_to_ids(tokens[1:-1]))
        #print(type(sent))
        tokeniz = tokenizer(sent, return_tensors=return_tensors)
        #print(tokenizer.convert_tokens_to_string(tokeniz))
        #print(tokeniz)
        tokeniz["labels"] = fin_tags
        print(fin_tags)
        #print('input_ids : ', len(tokeniz['input_ids']))
        #print('input_ids : ', tokeniz['input_ids'])
        #print(sent)
        #print('labels :', len(tokeniz['labels']))
        #print('am : ', len(tokeniz['attention_mask']))
        tokenizeds += [tokeniz]
    
    return tokenizeds

tokenizeds_dev = make_dataset(dataset_dev)
tokenizeds_train = make_dataset(dataset_train)
tokenizeds_test = make_dataset(dataset_test, return_tensors="pt")

[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [51]:
label_list = ["0", "1"]

In [52]:
id2label = {
    0: "0",
    1: "1"
}

label2id = {
    "0": 0,
    "1": 1
}

In [53]:
model = AutoModelForTokenClassification.from_pretrained(
        "distilbert/distilbert-base-uncased", 
        num_labels=2, 
        id2label=id2label, 
        label2id=label2id)
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [54]:
import evaluate

seqeval = evaluate.load("seqeval")

In [55]:
def compute_metrics(p):

    predictions, labels = p

    predictions = np.argmax(predictions, axis=2)

    true_predictions = [

        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]

        for prediction, label in zip(predictions, labels)

    ]

    true_labels = [

        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]

        for prediction, label in zip(predictions, labels)

    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)

    return {

        "precision": results["overall_precision"],

        "recall": results["overall_recall"],

        "f1": results["overall_f1"],

        "accuracy": results["overall_accuracy"],

    }

In [56]:
training_args = TrainingArguments(
    output_dir="./models/distilbert-v0",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenizeds_train,
    eval_dataset=tokenizeds_dev,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

  0%|          | 0/58 [00:00<?, ?it/s]

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


  0%|          | 0/7 [00:00<?, ?it/s]

/home/labicquette/.local/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/labicquette/.local/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 1 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/labicquette/.local/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/labicquette/.local/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/labicq

{'eval_loss': 0.1364659070968628, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_f1': 0.0, 'eval_accuracy': 0.9701447753356838, 'eval_runtime': 30.8638, 'eval_samples_per_second': 0.875, 'eval_steps_per_second': 0.227, 'epoch': 1.0}


  0%|          | 0/7 [00:00<?, ?it/s]

/home/labicquette/.local/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/labicquette/.local/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 1 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/labicquette/.local/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/labicquette/.local/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/labicq

{'eval_loss': 0.13515394926071167, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_f1': 0.0, 'eval_accuracy': 0.9701447753356838, 'eval_runtime': 29.5459, 'eval_samples_per_second': 0.914, 'eval_steps_per_second': 0.237, 'epoch': 2.0}
{'train_runtime': 912.2257, 'train_samples_per_second': 0.248, 'train_steps_per_second': 0.064, 'train_loss': 0.1469520535962335, 'epoch': 2.0}


TrainOutput(global_step=58, training_loss=0.1469520535962335, metrics={'train_runtime': 912.2257, 'train_samples_per_second': 0.248, 'train_steps_per_second': 0.064, 'train_loss': 0.1469520535962335, 'epoch': 2.0})

In [57]:
print(tokenizeds_dev[0].keys())

dict_keys(['input_ids', 'attention_mask', 'labels'])


In [65]:
model = DistilBertForTokenClassification.from_pretrained("./models/distilbert-v0/checkpoint-58")
labels = []
tok_labels = []
for x in tokenizeds_test:
    #print(x['input_ids'])
    inputs = {}
    #print(x.keys())
    inputs['input_ids'] = x['input_ids']
    inputs['attention_mask'] = x['attention_mask']
    with torch.no_grad():
        logits = model(**inputs).logits
    #print(logits)
    predicted_token_class_ids = logits.argmax(-1)
    print(np.unique(predicted_token_class_ids))
    predicted_tokens_classes = [model.config.id2label[t.item()] for t in predicted_token_class_ids[0]]
    labels += predicted_token_class_ids.tolist()
    print(np.unique(labels[-1]))
    tok_labels += [x['labels']]



[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]


In [59]:
print(len(tok_labels))
print(len(tokenizeds_test))
print(len(labels))

22
22
22


In [74]:
f_labels = [] 
for l in labels:
    f_labels += [l]
print(f_labels)
classification_report(labels, tok_labels)
#for i,j in zip(tok_labels, labels):
    
    #lab = np.where(j == 1)[0]
    #toks_lab = np.where(i == 1)[0]
    #compute_metrics((j, i))
    #evaluate_indices(lab, toks_lab)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


ValueError: You appear to be using a legacy multi-label data representation. Sequence of sequences are no longer supported; use a binary array or sparse matrix instead - the MultiLabelBinarizer transformer can convert to this format.